# CLARA FINE-TUNING - Fixed for Mistral & Phi-3 (v3)

**Fixes:**
- Separate training cells for Mistral and Phi-3
- Phi-3 gradient checkpointing compatibility fix
- Conservative hyperparameters

**Strategy:**
- **Personality** (warmth, playful, formal, encouragement) → **Mistral 7B**
- **Knowledge** (medical, coding, teaching, quantum) → **Phi-3-mini**

## Cell 1: Setup

In [ ]:
!pip install -q transformers==4.40.0 datasets accelerate wandb bitsandbytes
!pip install -q peft trl sentencepiece

import wandb
wandb.login()

from google.colab import drive
drive.mount('/content/drive')

import torch
import transformers
print(f"✓ Setup complete!")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Cell 2: Configuration - CHANGE DIMENSION HERE!

In [ ]:
# ============================================================
# WHICH DIMENSION ARE YOU TRAINING?
# ============================================================

DIMENSION = "medical"  # <-- CHANGE THIS EACH RUN

# ============================================================
# AUTO-CONFIGURATION
# ============================================================
PERSONALITY_DIMS = ["warmth", "playful", "formal", "encouragement"]
KNOWLEDGE_DIMS = ["medical", "coding", "teaching", "quantum"]

if DIMENSION in PERSONALITY_DIMS:
    BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"
    DATA_PATH = f"/content/drive/MyDrive/Lily/training_data/{DIMENSION}_training.json"
    model_prefix = "mistral"
    TRAINING_TYPE = "Personality"
    USE_PHI3 = False
elif DIMENSION in KNOWLEDGE_DIMS:
    BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"
    DATA_PATH = f"/content/drive/MyDrive/Lily/training_data/{DIMENSION}_knowledge.json"
    model_prefix = "phi3"
    TRAINING_TYPE = "Knowledge"
    USE_PHI3 = True
else:
    raise ValueError(f"Unknown dimension: {DIMENSION}")

OUTPUT_DIR = f"/content/drive/MyDrive/Lily/models/{model_prefix}_{DIMENSION}"

# ============================================================
# TRAINING CONFIG - CONSERVATIVE
# ============================================================
EPOCHS = 1
BATCH_SIZE = 4
GRADIENT_ACCUM = 4
LEARNING_RATE = 5e-5
MAX_LENGTH = 512
MAX_GRAD_NORM = 1.0

print("=" * 60)
print(f"🎯 TRAINING: {DIMENSION.upper()}")
print("=" * 60)
print(f"  Type:   {TRAINING_TYPE}")
print(f"  Model:  {BASE_MODEL}")
print(f"  Phi-3:  {USE_PHI3}")
print(f"  LR:     {LEARNING_RATE}")
print(f"  Output: {OUTPUT_DIR}")
print("=" * 60)

if USE_PHI3:
    print("\n⚠️  PHI-3 DETECTED: Run Cell 9-Phi (skip Cell 9-Mistral)")
else:
    print("\n⚠️  MISTRAL DETECTED: Run Cell 9-Mistral (skip Cell 9-Phi)")

## Cell 3: Check GPU

In [ ]:
!nvidia-smi

import torch
if torch.cuda.is_available():
    print(f"\n✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    raise RuntimeError("No GPU!")

## Cell 4: Load Data

In [ ]:
import json
import os
from datasets import Dataset

if not os.path.exists(DATA_PATH):
    print(f"❌ File not found: {DATA_PATH}")
    data_dir = "/content/drive/MyDrive/Lily/training_data"
    if os.path.exists(data_dir):
        print("\nAvailable:")
        for f in sorted(os.listdir(data_dir)):
            print(f"  - {f}")
    raise FileNotFoundError(DATA_PATH)

with open(DATA_PATH) as f:
    raw_data = json.load(f)

print(f"✓ Loaded {len(raw_data)} examples")

def format_data(examples, dimension, training_type):
    formatted = []
    for ex in examples:
        if training_type == "Personality":
            if 'neutral' in ex and 'high' in ex:
                formatted.append({
                    "instruction": f"Rewrite this with high {dimension}: {ex['neutral']}",
                    "response": ex['high']
                })
            if 'neutral' in ex and 'low' in ex:
                formatted.append({
                    "instruction": f"Rewrite this with low {dimension}: {ex['neutral']}",
                    "response": ex['low']
                })
        else:
            if 'question' in ex and 'answer' in ex:
                formatted.append({
                    "instruction": ex['question'],
                    "response": ex['answer']
                })
    return formatted

formatted = format_data(raw_data, DIMENSION, TRAINING_TYPE)
print(f"✓ Formatted: {len(formatted)} examples")

dataset = Dataset.from_list(formatted)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(f"  Train: {len(dataset['train'])}, Val: {len(dataset['test'])}")

## Cell 5: Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading {BASE_MODEL}...")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager",
    use_cache=False  # Important for training
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✓ Loaded: {model.num_parameters():,} params")

## Cell 6: Configure LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Different target modules for different models
if USE_PHI3:
    target_modules = ["qkv_proj", "o_proj"]  # Phi-3 uses combined qkv
else:
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✓ LoRA configured")
print(f"  Target modules: {target_modules}")
print(f"  Trainable: {trainable:,} ({100*trainable/total:.2f}%)")

## Cell 7: Tokenize

In [ ]:
def create_prompt(instruction, response=""):
    if response:
        return f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
    return f"### Instruction:\n{instruction}\n\n### Response:\n"

def tokenize_fn(examples):
    prompts = [create_prompt(i, r) for i, r in zip(examples['instruction'], examples['response'])]
    tokenized = tokenizer(prompts, truncation=True, max_length=MAX_LENGTH, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenizing...")
tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=dataset['train'].column_names)
print("✓ Done")

## Cell 8: Training Config

In [ ]:
from transformers import TrainingArguments
import os

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Phi-3 needs gradient checkpointing disabled due to cache issues
use_gradient_checkpointing = not USE_PHI3

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Training
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM,
    
    # Optimizer
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    
    # Precision
    bf16=True,
    
    # Logging
    logging_steps=10,
    logging_first_step=True,
    
    # Saving
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    
    # Eval
    eval_strategy="steps",
    eval_steps=25,
    
    # W&B
    report_to="wandb",
    run_name=f"clara-{model_prefix}-{DIMENSION}-v3",
    
    # Stability - KEY DIFFERENCE FOR PHI-3
    gradient_checkpointing=use_gradient_checkpointing,
    max_grad_norm=MAX_GRAD_NORM,
)

print("✓ Training config")
print(f"  Gradient checkpointing: {use_gradient_checkpointing}")
print(f"  LR: {LEARNING_RATE}")

## Cell 9-Mistral: Train Mistral (Personality)

**Run this cell for:** warmth, playful, formal, encouragement

**Skip this cell for:** medical, coding, teaching, quantum

In [ ]:
# ============================================================
# MISTRAL TRAINING (with gradient checkpointing)
# ============================================================

if USE_PHI3:
    print("⚠️  SKIP THIS CELL - You're training Phi-3!")
    print("   Run Cell 9-Phi instead.")
else:
    from transformers import Trainer, DataCollatorForLanguageModeling
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized['train'],
        eval_dataset=tokenized['test'],
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    )
    
    print("=" * 60)
    print(f"🚀 TRAINING MISTRAL: {DIMENSION.upper()}")
    print("=" * 60)
    print(f"  Watch: https://wandb.ai/chris_hartline/clara-deng-research")
    print("=" * 60)
    
    trainer.train()
    
    print("\n✓ Mistral training complete!")

## Cell 9-Phi: Train Phi-3 (Knowledge)

**Run this cell for:** medical, coding, teaching, quantum

**Skip this cell for:** warmth, playful, formal, encouragement

In [ ]:
# ============================================================
# PHI-3 TRAINING (without gradient checkpointing)
# ============================================================

if not USE_PHI3:
    print("⚠️  SKIP THIS CELL - You're training Mistral!")
    print("   Run Cell 9-Mistral instead.")
else:
    from transformers import Trainer, DataCollatorForLanguageModeling
    
    # Ensure cache is disabled for training
    model.config.use_cache = False
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized['train'],
        eval_dataset=tokenized['test'],
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    )
    
    print("=" * 60)
    print(f"🚀 TRAINING PHI-3: {DIMENSION.upper()}")
    print("=" * 60)
    print(f"  Note: Gradient checkpointing DISABLED for Phi-3 compatibility")
    print(f"  Watch: https://wandb.ai/chris_hartline/clara-deng-research")
    print("=" * 60)
    
    trainer.train()
    
    print("\n✓ Phi-3 training complete!")

## Cell 10: Save Model

In [ ]:
print(f"Saving to: {OUTPUT_DIR}")

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Verify save
required_files = ['adapter_config.json', 'adapter_model.safetensors']
saved_files = os.listdir(OUTPUT_DIR)

print(f"\nSaved files:")
for f in saved_files:
    size = os.path.getsize(os.path.join(OUTPUT_DIR, f)) / 1e6
    print(f"  - {f} ({size:.1f} MB)")

# Check required files exist
if 'adapter_config.json' in saved_files:
    print("\n✅ Model saved correctly!")
else:
    print("\n❌ WARNING: adapter_config.json missing!")

## Cell 11: Test Model

In [ ]:
print("\n" + "=" * 60)
print("TESTING TRAINED MODEL")
print("=" * 60)

# Enable cache for inference
model.eval()
model.config.use_cache = True

# Disable gradient checkpointing for inference
if hasattr(model, 'gradient_checkpointing_disable'):
    model.gradient_checkpointing_disable()
if hasattr(model.base_model, 'gradient_checkpointing_disable'):
    model.base_model.gradient_checkpointing_disable()

def generate_response(prompt, max_new_tokens=100):
    full_prompt = create_prompt(prompt)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    return response

# Test prompts
if TRAINING_TYPE == "Personality":
    test_prompts = [
        f"Rewrite this with high {DIMENSION}: I can help you with that.",
        f"Rewrite this with high {DIMENSION}: That's correct.",
    ]
else:
    test_prompts = [
        f"Explain a basic concept in {DIMENSION}.",
        f"What should a beginner know about {DIMENSION}?",
    ]

print("\nResults:\n")
all_ok = True
for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    response = generate_response(prompt)
    print(f"Response: {response}\n")
    
    if "Question:" in response or "}}" in response or "QR" in response:
        print("⚠️ WARNING: Garbled output!")
        all_ok = False
    print("-" * 40)

if all_ok:
    print("\n✅ Model looks good!")
else:
    print("\n⚠️ Model may need retraining")

## Cell 12: Progress & Next Steps

In [ ]:
wandb.finish()

import os
models_dir = "/content/drive/MyDrive/Lily/models"

print("\n" + "=" * 60)
print("TRAINING PROGRESS")
print("=" * 60)

all_dims = PERSONALITY_DIMS + KNOWLEDGE_DIMS
completed = 0

for dim in all_dims:
    prefix = "mistral" if dim in PERSONALITY_DIMS else "phi3"
    path = f"{models_dir}/{prefix}_{dim}"
    
    # Check if properly saved
    if os.path.exists(path) and os.path.exists(f"{path}/adapter_config.json"):
        icon = "✅"
        completed += 1
    elif os.path.exists(path):
        icon = "⚠️"  # Exists but may be incomplete
    else:
        icon = "⬜"
    
    model_type = "Mistral" if dim in PERSONALITY_DIMS else "Phi-3"
    print(f"  {icon} {dim} ({model_type})")

print(f"\nCompleted: {completed}/8")

if completed < 8:
    remaining = []
    for d in all_dims:
        prefix = "mistral" if d in PERSONALITY_DIMS else "phi3"
        path = f"{models_dir}/{prefix}_{d}"
        if not os.path.exists(path) or not os.path.exists(f"{path}/adapter_config.json"):
            remaining.append(d)
    
    if remaining:
        next_dim = remaining[0]
        next_type = "Mistral" if next_dim in PERSONALITY_DIMS else "Phi-3"
        print(f"\n👉 Next: DIMENSION = '{next_dim}' ({next_type})")
        print(f"   Change Cell 2 and run all cells again.")
else:
    print("\n🎉 ALL TRAINING COMPLETE!")
    print("\nNext: Run mergekit to combine models into Clara!")